In [1]:
# importing essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [2]:
# importing dataset
data=pd.read_csv('/content/drive/MyDrive/BTC/btc.csv')
data.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# duplicate data as df
df=data.copy()

In [4]:
# creating date column
df['date'] = pd.to_datetime(df['Timestamp'],unit='s').dt.date
df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,date
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39,2011-12-31
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-31


In [5]:
# grouping data
group = df.groupby('date')
real_price = group['Weighted_Price'].mean()

In [6]:
real_price

date
2011-12-31        4.471603
2012-01-01        4.806667
2012-01-02        5.000000
2012-01-03        5.252500
2012-01-04        5.208159
                  ...     
2021-03-27    55193.357260
2021-03-28    55832.958824
2021-03-29    56913.993819
2021-03-30    58346.912268
2021-03-31    58764.349363
Name: Weighted_Price, Length: 3376, dtype: float64

In [7]:
# setting train and test data
prediction_days = 30
train= real_price[:len(real_price)-prediction_days]
test= real_price[len(real_price)-prediction_days:]

In [8]:
# scaling the data
training_set = train.values
training_set = np.reshape(training_set, (len(training_set), 1))
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
training_set = sc.fit_transform(training_set)

In [9]:
# initialising x_train and y_train
X_train = training_set[0:len(training_set)-1]
y_train = training_set[1:len(training_set)]

In [10]:
# reshaping in 3-D because our layers need like this
X_train = np.reshape(X_train, (len(X_train), 1,1))

In [11]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# Initialising the RNN
regressor = Sequential()

# Adding the input layer and the LSTM layer
regressor.add(LSTM(units = 10, activation = 'relu', input_shape = (None, 1)))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, batch_size = 5, epochs = 100)

Epoch 1/100
669/669 [==============================] - 3s 2ms/step - loss: 0.0081
Epoch 2/100
669/669 [==============================] - 1s 2ms/step - loss: 0.0017
Epoch 3/100
669/669 [==============================] - 1s 2ms/step - loss: 1.5831e-04
Epoch 4/100
669/669 [==============================] - 1s 2ms/step - loss: 1.2200e-04
Epoch 5/100
669/669 [==============================] - 1s 2ms/step - loss: 1.0077e-04
Epoch 6/100
669/669 [==============================] - 1s 2ms/step - loss: 7.9630e-05
Epoch 7/100
669/669 [==============================] - 1s 2ms/step - loss: 6.6410e-05
Epoch 8/100
669/669 [==============================] - 1s 2ms/step - loss: 5.4137e-05
Epoch 9/100
669/669 [==============================] - 1s 2ms/step - loss: 4.9243e-05
Epoch 10/100
669/669 [==============================] - 1s 2ms/step - loss: 4.4535e-05
Epoch 11/100
669/669 [==============================] - 1s 2ms/step - loss: 4.3992e-05
Epoch 12/100
669/669 [==============================] - 1s 2

In [12]:
test[:10]

date
2021-03-02    48617.289321
2021-03-03    50429.449016
2021-03-04    49398.250399
2021-03-05    47717.643300
2021-03-06    48371.543538
2021-03-07    50218.216804
2021-03-08    50798.354212
2021-03-09    53980.847262
2021-03-10    55375.366241
2021-03-11    56272.596972
Name: Weighted_Price, dtype: float64

In [13]:
# Making the predictions
test_set = test.values
inputs = np.reshape(test_set, (len(test_set), 1))
inputs = sc.transform(inputs)
inputs = np.reshape(inputs, (len(inputs), 1, 1))
predicted_BTC_price = regressor.predict(inputs)


In [14]:
# converting to normal numbers
predicted_BTC_price = sc.inverse_transform(predicted_BTC_price)

In [17]:
predicted_BTC_price.shape

(30, 1)

In [18]:
pred=pd.DataFrame(predicted_BTC_price)

In [19]:
pred.head()

,0
0,49084.726562
1,50819.332031
2,49832.859375
3,48221.816406
4,48849.128906


In [34]:
test_set
final=pd.DataFrame({'actual':test_set,'date':test.index})
final

,actual,date
0,48617.289321,2021-03-02
1,50429.449016,2021-03-03
2,49398.250399,2021-03-04
3,47717.643300,2021-03-05
4,48371.543538,2021-03-06
5,50218.216804,2021-03-07
6,50798.354212,2021-03-08
7,53980.847262,2021-03-09
8,55375.366241,2021-03-10
9,56272.596972,2021-03-11


In [35]:
pred

,0
0,49084.726562
1,50819.332031
2,49832.859375
3,48221.816406
4,48849.128906
5,50617.398438
6,51171.843750
7,54204.066406
8,55527.488281
9,56377.207031


In [ ]:

final=pd.merge(final,pred,on=final.index)

In [39]:
final.head()

,key_0,actual,date,0
0,0,48617.289321,2021-03-02,49084.726562
1,1,50429.449016,2021-03-03,50819.332031
2,2,49398.250399,2021-03-04,49832.859375
3,3,47717.643300,2021-03-05,48221.816406
4,4,48371.543538,2021-03-06,48849.128906


In [40]:
final=final.drop(['key_0'],axis=1)
final = final.rename(columns={0: 'predicted'})
final

,actual,date,predicted
0,48617.289321,2021-03-02,49084.726562
1,50429.449016,2021-03-03,50819.332031
2,49398.250399,2021-03-04,49832.859375
3,47717.643300,2021-03-05,48221.816406
4,48371.543538,2021-03-06,48849.128906
5,50218.216804,2021-03-07,50617.398438
6,50798.354212,2021-03-08,51171.843750
7,53980.847262,2021-03-09,54204.066406
8,55375.366241,2021-03-10,55527.488281
9,56272.596972,2021-03-11,56377.207031


In [55]:
import plotly.express as px
import plotly.graph_objects as go

# fig=px.line(final,x='date',y='predicted')
# fig.add_scatter(x=final['date'], y=final['actual'])
# fig.show()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=final.date,
    y=final.predicted,
    name="Predicted"       # this sets its legend entry
))


fig.add_trace(go.Scatter(
    x=final.date,
    y=final.actual,
    name="Original"
))

fig.update_layout(
    title="Plot Title",
    xaxis_title="Date",
    yaxis_title="Price",
    )


fig.show()